In [1]:
import pandas as pd
rbns_status = pd.read_csv('/tscc/nfs/home/hsher/projects/rbns_thermo/RBNS_status.csv', names = ['rbp', 'status'])
rbns_status['status'] = rbns_status['status'].fillna('NAN').str.upper()
available_rbps = rbns_status.loc[rbns_status['status']!='FAIL']['rbp'].tolist()

In [2]:
from collections import defaultdict
import os
rbp_to_file = defaultdict(list)
for f in os.listdir('/tscc/nfs/home/hsher/projects/rbns_thermo/Burge_RBNS/'):
    rbp = f.split('_')[0]
    if rbp in available_rbps:
        rbp_to_file[rbp].append(os.path.join('/tscc/nfs/home/hsher/projects/rbns_thermo/Burge_RBNS/',f))

In [3]:
from pathlib import Path
indir = Path('/tscc/nfs/home/hsher/ps-yeolab5/selex_motif/linear/')
rbp_to_selex = {}
for file in indir.glob('*.motif.csv'):
    rbp_to_selex[file.name.split('.')[0]]=file
indir = Path('/tscc/nfs/home/hsher/ps-yeolab5/selex_motif/struct/')
for file in indir.glob('*.motif.csv'):
    rbp_to_selex[file.name.split('.')[0]]=file

In [12]:
import numpy as np
from scipy.stats import entropy
class Motif:
    def __init__(self, pwm, pseudocount = 0.000001):
        self.pwm = pwm
        
        self.filter_by_entropy()
        self.pwm += pseudocount

        # clean up index, make U to T (same as bedtools)
        
        self.pwm.columns = ['T' if i == 'U' else i for i in list(self.pwm.columns)]
        self.logodds_score()
    def filter_by_entropy(self, thres = 1):
        '''calculate entropy for each position '''
        m_entropy = np.apply_along_axis(arr = self.pwm.values, func1d = entropy, axis = 1)
        self.pwm = self.pwm.loc[m_entropy <thres]
        self.pwm.reset_index(inplace = True)
        self.pwm.drop('index', axis = 1, inplace = True)
    def score_samelen_seq(self, seq):
        ''' return log likelihood score of a sequence '''
        score = 0
        assert len(seq) == self.pwm.shape[0]
        for index, s in enumerate(seq):
            score += np.log(self.pwm.loc[index, s]) # the higher the better (higher prob)
        return score
    def score_entire_seq(self, seq):
        ''' return the scores of every kmer in sequence '''
        k = self.pwm.shape[0]
        scores = []
        for i in range(len(seq)-k+1):
            substr = seq[i:i+k]
            substr_score = self.score_samelen_seq(substr)
            scores.append(substr_score)
        return scores
    def max_scoring_substring(self, seq):
        scores = self.score_entire_seq(seq)
        max_score = max(scores)
        index_start = scores.index(max_score)
        index_end = index_start+ self.pwm.shape[0]
        return max_score, index_start, index_end
    def logodds_score(self):
        max_pos = self.pwm.idxmax(axis = 1)
        s = 0
        for i, seq in zip(self.pwm.index, max_pos):
            s += np.log(self.pwm.loc[i, seq]/0.25)
        self.logodds_score = s
        self.sequence = ''.join(max_pos)
        
        

In [9]:
def motif2homer(m, name, out_handle):
    string = f'>{m.sequence}\t{name}_{m.sequence}\t{m.logodds_score}\n'
    
    out_handle.write(string)
    m.pwm[['A', 'C', 'G', 'T']].to_csv(out_handle, index = False, header = False, float_format='%.3f', sep = '\t')

In [20]:
pd.read_csv(rbp_to_file[key][0], index_col = 0, sep = '\t', skiprows = 1)

,A,C,G,U
PO,,,,
0,0.050905,0.050805,0.847485,0.050805
1,0.000000,1.000000,0.000000,0.000000
2,0.406341,0.593659,0.000000,0.000000
3,0.192619,0.197720,0.609661,0.000000
4,0.000000,1.000000,0.000000,0.000000
5,0.046905,0.859386,0.046905,0.046805


In [21]:
with open('all_RBP_SELEX_RBNS.homer', 'w') as f:
    for key in rbp_to_selex.keys():
        m=Motif(pd.read_csv(rbp_to_selex[key], index_col = 0))
        motif2homer(m, key,f)
    for key in rbp_to_file.keys():
        df = pd.read_csv(rbp_to_file[key][0], index_col = 0, sep = '\t', skiprows = 1)
        df.index.name = 'index'
        m=Motif(df)
        motif2homer(m, key,f)

/tscc/nfs/home/hsher/miniconda3/envs/my_metadensity/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [22]:
# from plot_params import *
# import pandas as pd
# df=pd.read_csv(outdir / 'Mouse_annotated.csv.gz')

In [23]:
# n_c = df['subseq'].str.count('C').sum()
# n_g = df['subseq'].str.count('G').sum()
# n_t = df['subseq'].str.count('T').sum()
# n_a = df['subseq'].str.count('A').sum()

In [ ]:
# m.max_scoring_substring(df.iloc[0]['subseq'])

In [ ]:
# df.iloc[:50]['subseq'].apply(m.max_scoring_substring)

module load homer
cd /tscc/nfs/home/hsher/ps-yeolab5/karen_data
<!-- findMotifs.pl utr3_forefround.fa fasta utr3_homer_results/ -fasta utr3_backfround.fa \
    -rna -nofacts -->

# mouse
findMotifs.pl all.fa fasta test_homer_find -find /tscc/nfs/home/hsher/projects/karen_LLM/all_RBP_SELEX_RBNS.homer -rna > homer_output.tsv

# human
findMotifs.pl all_human.fa fasta test_homer_find_human -find /tscc/nfs/home/hsher/projects/karen_LLM/all_RBP_SELEX_RBNS.homer -rna > homer_output_human.tsv
<!-- findMotifs.pl all.fa fasta test_homer_find -find ~/scratch/ENCODE3_HepG2/output/homer/finemapped_results/RBFOX2_HepG2_ENCSR987FTF/homerMotifs.all.motifs -rna > ~/scratch/testhomerfind.txt
 -->

findMotifs.pl all_human_30bp.fa fasta test_homer_find_human -find /tscc/nfs/home/hsher/projects/karen_LLM/all_RBP_SELEX_RBNS.homer -rna > homer_output_human.tsv